## FASE 1. Extracción de datos de películas

In [58]:
# importamos todas las librerias necesarias para realizar el ejercicio

import requests
import pandas as pd
import mysql.connector
from mysql.connector import Error
import numpy as np

In [19]:
# extraemos los datos de la API y la guardamos en una variable (en formato JSON)

url = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

datos = requests.get(url)
datos.status_code
datos.content

datos_pelis = datos.json()

## FASE 2. Creación de la base de datos

In [20]:
# Iniciamos la connexion con SQL ---> Se deberá modificar el password acorde con el del usuario que ejecute el codigo. 
try:
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='AlumnaAdalab',   
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)


Conexión exitosa


In [21]:
# Creamos la base de datos 

try:
    mycursor = cnx.cursor(buffered=True)
    query = "CREATE DATABASE IF NOT EXISTS films_ejercicio_2_version3"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Query exitosa


In [ ]:
# para crear las tablas primero debemos revisar los datos del API y analizar que tablas y columnas debemos crear.
# Para este ejercicio, he decidido crear 4 tablas: 1. tabla de peliculas 2. tabla de generos 3. tabla de subtitulos 4. tabla intermedia peliculas_subtitulos

In [ ]:
mycursor.execute("USE films_ejercicio_2_version3")

query = '''CREATE TABLE genero (
    id_genero INT PRIMARY KEY AUTO_INCREMENT,
    genero VARCHAR(45) NOT NULL  
);'''

mycursor.execute(query)

In [ ]:
mycursor.execute("USE films_ejercicio_2_version3")
mycursor.execute("DROP TABLE IF EXISTS subtitulos")

query = '''CREATE TABLE subtitulos (
    id_idioma INT PRIMARY KEY AUTO_INCREMENT,
    idioma VARCHAR(5) NOT NULL  
);'''

mycursor.execute(query)

In [24]:

mycursor.execute("USE films_ejercicio_2_version3")

mycursor.execute("DROP TABLE IF EXISTS films") #añado el drop table if exists por si tengo que volver a ejecutar el codigo con alguna modificacion

query = '''CREATE TABLE films (
    id_film INT PRIMARY KEY AUTO_INCREMENT,
    titulo VARCHAR(255) NOT NULL,
    año INT,
    duracion FLOAT,
    id_genero INT, 
    adultos TINYINT(1),
    FOREIGN KEY (id_genero) REFERENCES genero(id_genero)


);'''

mycursor.execute(query)

In [33]:
mycursor.execute("USE films_ejercicio_2_version3")
mycursor.execute("DROP TABLE IF EXISTS peliculas_subtitulos")
query = '''CREATE TABLE peliculas_subtitulos (
    id_film INT,
    id_idioma INT,
    PRIMARY KEY (id_film, id_idioma),
    FOREIGN KEY (id_film) REFERENCES films(id_film),
    FOREIGN KEY (id_idioma) REFERENCES subtitulos(id_idioma)   
);'''

mycursor.execute(query)

## Fase 3. Inserción de los Datos en la Base de datos

## Insertamos datos en tabla subtitulos

In [ ]:
# busco en la lista datos_pelis que subtitulos hay

datos_subtitulos = []
for i in datos_pelis:
    for subtitulos in i['subtitulos']:
        datos_subtitulos.append(subtitulos)

# a continuación paso la información a set y luego de nuevo a lista para que me deje solo los valores únicos. 
lista_subtitulos = list(set(datos_subtitulos))


['ko', 'pt', 'en', 'fr', 'it', 'es', 'jp', 'de']

In [ ]:
#Una vez tenemos los valores únicos, insertamos los datos a la tabla:
for i in lista_subtitulos:
    query = "INSERT INTO subtitulos (idioma) VALUES (%s)"
    mycursor.execute(query, (i,))

## Insertamos datos en tabla genero

In [ ]:
#Encontramos los valores que existen en genero
datos_genero = []
for i in datos_pelis:
    datos_genero.append(i['genero'])

# Nos quedamos con los valores únicos. Creando una set y luego lo pasamos a lista de nuevo.
lista_genero = list(set(datos_genero))


In [ ]:
#Una vez tenemos los valores únicos, insertamos los datos a la tabla:
for i in lista_genero:
    query = "INSERT INTO genero (genero) VALUES (%s)"
    mycursor.execute(query, (i,))

In [ ]:
cnx.commit()  #guardamos los datos a SQL (necesario cuando se hacen inserts)

## Insertamos datos en tabla films

In [31]:
datos_limpios_films = []

for peli in datos_pelis:
    # Obtener id_genero
    sql = "SELECT id_genero FROM genero WHERE genero = %s"
    mycursor.execute(sql, (peli['genero'],))
    id_genero = mycursor.fetchone()[0]

    # Crear la fila lista para insertar
    fila = (
        peli['titulo'],
        peli['año'],
        peli['duracion'],
        id_genero,               
        int(peli['adultos'])
    )

    datos_limpios_films.append(fila)


mycursor.executemany("""
    INSERT INTO films (titulo, año, duracion, id_genero, adultos)
    VALUES (%s, %s, %s, %s, %s)
""", datos_limpios_films)
    
cnx.commit()

## Insertar datos en tabla intermedia pelicula_subtitulos

In [34]:
mycursor.execute("USE films_ejercicio_2_version3")

for peli in datos_pelis:

    # Buscar id_film por título (porque el título es único en tu dataset)
    mycursor.execute("SELECT id_film FROM films WHERE titulo = %s", (peli['titulo'],))
    id_film = mycursor.fetchone()[0]

    # Insertar subtítulos en la tabla puente
    for sub in peli['subtitulos']:
        mycursor.execute(
            "SELECT id_idioma FROM subtitulos WHERE idioma = %s", 
            (sub,)
        )
        id_idioma = mycursor.fetchone()[0]

        mycursor.execute("""
            INSERT INTO peliculas_subtitulos (id_film, id_idioma)
            VALUES (%s, %s)
        """, (id_film, id_idioma))

cnx.commit()

In [ ]:
#Una vez terminamos el flujo de trabajo de inserción. Si dejamos de trabajar usando la connexion con SQL. Podriamos cerrarla.
mycursor.close()
cnx.close()
print('Conexión cerrada')

## FASE 4. Obtener información a partir de los datos.



In [ ]:
# Si hemos cerrado la connexion con SQL annteriormente. Ahora debemos iniciarla de nuevo ---> Se deberá modificar el password acorde con el del usuario que ejecute el codigo. 
# Dado que hemos creado el Database en el apartado anterior. podemos ya abrir 
try:
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='AlumnaAdalab',    
        database = 'films_ejercicio_2_version3'
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)


In [ ]:
mycursor = cnx.cursor()
mycursor.execute("USE films_ejercicio_2_version3") # no haría falta si en el punto anterior hemos realizado la connexión ya usando este database

¿Cuántas películas tienen una duración superior a 120 minutos?

In [47]:
query = """
SELECT COUNT(*)
FROM films
WHERE duracion > 120
"""
mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[(59,)]

¿Cuántas películas incluyen subtítulos en español?

In [48]:
query = """
SELECT COUNT(*)
FROM peliculas_subtitulos ps
INNER JOIN subtitulos s ON s.id_idioma = ps.id_idioma
WHERE idioma LIKE "%ES%";
"""
mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[(100,)]

¿Cuántas películas tienen contenido adulto?

In [49]:
query = """
SELECT COUNT(*)
FROM films
WHERE adultos = 1;
"""
mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[(47,)]

¿Cuál es la película más antigua registrada en la base de datos?

In [38]:
query = """
SELECT titulo, año
FROM films
ORDER BY año
LIMIT 1
"""
mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[('Citizen Kane', 1941)]

Muestra el promedio de duración de las películas agrupado por género.

In [50]:
query = """
SELECT ROUND(AVG(duracion),2) AS promedio_duracion, genero
FROM films
RIGHT JOIN genero ON genero.id_genero = films.id_genero
GROUP BY genero;
"""

#En este caso muestro el resultado en un data frame asi vemos las columnas y sus cabeceras
df = pd.read_sql(query, cnx)
df


C:\Users\laura\AppData\Local\Temp\ipykernel_20576\375698593.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


,promedio_duracion,genero
0,159.80,Fantasía
1,103.00,Animación
2,126.26,Drama
3,121.67,Thriller
4,133.00,Aventura
5,136.31,Ciencia ficción
6,154.29,Crimen
7,139.44,Acción
8,120.00,Suspense
9,166.50,Western


¿Cuántas películas por año se han registrado en la base de datos? Ordena de mayor a menor.

In [51]:
query = """
SELECT count(id_film) AS total_peliculas, año
FROM films
GROUP BY año
ORDER BY total_peliculas DESC
"""


df = pd.read_sql(query, cnx)
df


C:\Users\laura\AppData\Local\Temp\ipykernel_20576\2165272846.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


,total_peliculas,año
0,5,2001
1,4,2013
2,4,1994
3,4,2008
4,4,1999
5,4,2017
6,3,2010
7,3,1998
8,3,2014
9,3,2000


¿Cuál es el año con más películas en la base de datos

In [53]:
query = """
SELECT año
FROM films
GROUP BY año
ORDER BY COUNT(*) DESC
LIMIT 1;
"""


mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[(2001,)]

Obtén un listado de todos los géneros y cuántas películas corresponden a cada uno.

In [56]:
query = """
SELECT genero, count(id_film) AS Total_peliculas
FROM films
RIGHT JOIN genero ON genero.id_genero = films.id_genero
GROUP BY genero
ORDER BY Total_peliculas DESC
"""


df = pd.read_sql(query, cnx)
df

C:\Users\laura\AppData\Local\Temp\ipykernel_20576\339514744.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


,genero,Total_peliculas
0,Drama,27
1,Ciencia ficción,13
2,Animación,9
3,Acción,9
4,Crimen,7
5,Terror,7
6,Thriller,6
7,Fantasía,5
8,Aventura,3
9,Biografía,3


Muestra todas las películas cuyo título contenga la palabra "Godfather" (puedes usar cualquier palabra).

In [57]:
query = """
SELECT titulo
FROM films
WHERE titulo LIKE '%GODFATHER%';
"""


mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[('The Godfather',), ('The Godfather Part II',)]